In [8]:
import pandas as pd

url = (
    "https://darts.isas.jaxa.jp/app/query/astroquery/sql.php?"
    "format=html&"
    "sql="
    "SELECT%0A"
    "observation_id%2Cobject_name%2Ccenter_ra%2Ccenter_dec%2C"
    "roll_angle%2Cobservation_start_time%2Cobservation_end_time%2C"
    "resolve_exposure%2Cxtend_exposure%2Cprocessing_date%2Cpublic_date%2C"
    "distribution_date%2Cprocessing_version%2Cdata_access_url%0A"
    "FROM%20xrism_master_data%0A"
    "where%20observation_start_time_mjd%20%3E%2060209%0A"
    "order%20by%20observation_start_time%20desc"
)

# observation_id を文字列として読み込む
df = pd.read_html(url, header=0, converters={"observation_id": str})[0]

# 最左列が行番号なら削除
if df.columns[0] == 0:
    df = df.drop(columns=df.columns[0])

# 列名整形
df.columns = [c.strip() for c in df.columns]
# ① obsid が 0 または 3 で始まる行
mask_obsid = df["observation_id"].astype(str).str.startswith(("0", "3"), na=False)

# ② ターゲット名のマスク（大小・空白無視にするため前処理）
targets   = ["perseus", "m87", "abell2029", "coma",
             "centaurus", "abell2319", "pks_0745"]

# object_name を小文字化 → 連続する空白を除去してから判定
name_norm = (
    df["object_name"]
      .astype(str)
      .str.lower()
      .str.replace(r"\s+", "", regex=True)
)

mask_target = name_norm.str.contains("|".join(targets))

# --- 条件を両方満たす行だけ抜き出す ------------------
df_sel = df[mask_obsid & mask_target].reset_index(drop=True)

# 大文字・小文字を無視して object_name でソート
df_sorted = (
    df_sel
    .sort_values(
        by="object_name",
        key=lambda col: col.str.lower()   # col.str.lower() で小文字化して比較
    )
    .reset_index(drop=True)
)

# 必要な列だけを表示して確認
print(
    df_sorted[
        ["observation_id", "object_name",
         "observation_start_time", "resolve_exposure"]
    ]
)

   observation_id        object_name observation_start_time  resolve_exposure
0       000151000   Abell2029_Center    2024-01-13 01:51:04            29.337
1       000149000   Abell2029_Center    2024-01-10 03:31:04            16.324
2       000147000        Abell2029_E    2024-01-09 00:41:04            23.026
3       000146000        Abell2029_N    2024-01-08 14:41:04            20.112
4       000150000       Abell2029_N1    2024-01-10 16:21:04           106.210
5       300053010       Abell2029_N2    2024-07-27 16:16:04           383.186
6       000152000       Abell2029_N2    2024-01-13 19:31:04            50.437
7       000148000       Abell2029_SW    2024-01-09 14:41:04            27.591
8       000103000          Abell2319    2023-10-19 23:51:04            97.672
9       000101000          Abell2319    2023-10-13 23:51:04           146.644
10      000153000       Abell2319_BS    2024-01-15 00:01:04            11.603
11      000113000       Abell2319_BS    2023-11-11 12:01:04     

In [14]:
import pandas as pd
from pathlib import Path

# ─────────────────── 0) データ取得は以前と同じ ───────────────────
df = pd.read_html(
    url,
    header=0,
    converters={"observation_id": str, "public_date": str}  # public_date を文字列で保持
)[0]
if df.columns[0] == 0:
    df = df.drop(columns=df.columns[0])
df.columns = [c.strip() for c in df.columns]

# ─────────────────── 1) 公開日フィルタ (2025-08-31) ───────────────────
#   - public_date 列は "YYYY-MM-DD …" 形式なので startswith が簡単
mask_pub = df["public_date"].str.startswith("2025-08-31", na=False)

# ─────────────────── 2) ターゲット名フィルタ (大文字小文字無視) ───────────────────
targets = [
    "perseus", "m87", "abell2029", "coma",
    "centaurus", "abell2319", "pks_0745"
]
name_norm = (
    df["object_name"]
      .astype(str)
      .str.lower()
      .str.replace(r"\s+", "", regex=True)
)
mask_tgt = name_norm.str.contains("|".join(targets))

# ─────────────────── 3) 両方満たす行だけ抽出 ───────────────────
df_sel = df[mask_pub & mask_tgt].reset_index(drop=True)

# ─────────────────── 4) アルファベット順ソート ───────────────────
df_sorted = (
    df_sel.sort_values(
        by="object_name",
        key=lambda col: col.str.lower()
    ).reset_index(drop=True)
)

# ─────────────────── 5) “ベース名”列を作り色付け ───────────────────
base_keys = [
    "abell2029", "abell2319", "centaurus", "coma",
    "m87", "perseus", "pks0745"
]

def extract_basename(name: str) -> str:
    norm = name.lower().replace(" ", "")
    for key in base_keys:
        if key in norm:
            return key
    return "other"

df_sorted["base_name"] = df_sorted["object_name"].apply(extract_basename)

palette = ["#FFCCCC", "#CCFFCC", "#CCCCFF", "#FFF2CC",
           "#D5E8D4", "#F8CECC", "#E1D5E7"]
color_map = {b: palette[i % len(palette)]
             for i, b in enumerate(df_sorted["base_name"].unique())}

def highlight_row(row):
    return [f"background-color: {color_map[row['base_name']]}"] * len(row)

styled = (
    df_sorted
      .style
      .apply(highlight_row, axis=1)
      .hide(axis="columns", subset=["base_name"])
)

# ─────────────────── 6) Excel 出力 ───────────────────
out = Path("/Users/keitatanaka/Dropbox/XRISM_PV_archive/xrism_egd_bybasename.xlsx")
styled.to_excel(out, index=False, engine="openpyxl")   # openpyxl を事前に pip install
print(f"→ {out} に保存しました")

→ /Users/keitatanaka/Dropbox/XRISM_PV_archive/xrism_egd_bybasename.xlsx に保存しました
